In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from splinter import Browser
from time import sleep

In [224]:
nba_url = 'https://www.spotrac.com/nfl/draft/'
year = '2017'

In [225]:
r = requests.get((nba_url + year))

In [226]:
soup = BeautifulSoup(r.text, 'html.parser')

In [67]:
table_headers = soup.find_all('th')[0:10]
column_names = []
for x in table_headers:
    column_names.append(x.text.strip())


In [68]:
df = pd.DataFrame(columns = column_names)#, index = [i for i in range(256)])

In [256]:
# table_data = soup.find_all('td')
# for i in range(10):
#     print(table_data[i].text)
# len(table_data)

In [233]:
cnt = -1
for i in range(len(table_data)):
    if(i % 10 == 0):
        cnt += 1
#     print(i)
    temp = table_data[i]
    temp_text = temp.text
    df.iat[cnt, i%10] = temp_text

In [153]:
df['year'] = year

In [155]:
df.to_csv(f'nba_draft_player_salaries_{year}.csv',index=False)

In [182]:
#sport argument accepts 'nba', 'nfl', 'mlb' as acceptable arguments
#year is the year that is requested
#rows_count is the number of rows that are to be pulled from the requested site
def scrape_salaries_nba(year):
    
    r = requests.get(f'https://www.spotrac.com/nba/draft/' + year)
    soup = BeautifulSoup(r.text, 'html.parser')
    table_data = soup.find_all('td')
    
    columns = ['pick', 'team', 'player', 'pos', 'height', 'weight', 'country', 'college', 'years', 'dollars', 'year cap hit']
    df = pd.DataFrame(columns = columns, index = [i for i in range(60)])
    
    cnt = -1
    for i in range(len(table_data)):
        if(i % 11 == 0):
            cnt += 1
        df.iat[cnt, i%11] = table_data[i].text
    
    df['year'] = year
    df.to_csv(f'nba_draft_player_salaries_{year}.csv',index=False)

In [241]:
r.status_code

200

In [242]:
def scrape_salaries(sport, year, rows, num_columns):
    
    #requests site specific to sport and selects all relevant data
    r = requests.get(f'https://www.spotrac.com/{sport}/draft/{year}/')
    
    if r.status_code == 200:

        soup = BeautifulSoup(r.text, 'html.parser')
        table_data = soup.find_all('td')
        table_headers = soup.find_all('th')[0:num_columns]

        #gets column names from table_headers on requested page
        column_names = []
        for x in table_headers:
            column_names.append(x.text)

        #created dataframe to store data
        df = pd.DataFrame(columns = column_names, index = [i for i in range(rows)])
        #loops through requested data and appends it to table in very smart manner
        cnt = -1
        for i in range(len(table_data)):
            if(i % len(table_headers) == 0):
                cnt += 1
            df.iat[cnt, i % len(table_headers)] = table_data[i].text
        #attaches year to dataframe and exports it as .csv
        df['year'] = year
        df.to_csv(f'{sport}_draft_player_salaries_{year}.csv',index=False)
    else:
        return 'Request not successful, Code: ' + r.status_code

In [254]:
scrape_salaries('nfl', '2005', 256, 10)

In [2]:
bb_year = 2010
bb_url = f'https://www.baseballamerica.com/draft-history/mlb-draft-database/#/?Year={bb_year}&Round=0&TeamId=0&Position=&SchoolName=&FourYearSchoolType=true&JuniorCollegeType=false&HighSchoolType=false&OtherSchoolType=false&StateOrCountry=&Signed=&SigningBonusMin=0&SigningBonusMax=0&PlayerName=&OverallNumber=0'

In [19]:
browser = Browser('chrome')

In [20]:
browser.visit(bb_url)

In [30]:
browser.find_by_css('button').last.text

'SEE MORE PLAYERS'

In [35]:
while browser.find_by_css('button').last.text == 'SEE MORE PLAYERS':
    try:
        browser.find_by_css('button').last.click()
    except:
        sleep(.5)

In [49]:
soup = BeautifulSoup(browser.html, 'html.parser')
table_headers = soup.find_all('th')[0:12]

In [48]:
for x in table_headers:
    print(x.text.strip())

Year
RD
Pick
Team
Pos
Player
School
Type
ST
Signed
Bonus
Reports


In [86]:
def scrape_mlb(years):
    browser = Browser('chrome')
    
    for year in years:
        print('starting: ' + str(year))
        url = f'https://www.baseballamerica.com/draft-history/mlb-draft-database/#/?Year={year}&Round=0&TeamId=0&Position=&SchoolName=&FourYearSchoolType=false&JuniorCollegeType=false&HighSchoolType=false&OtherSchoolType=false&StateOrCountry=&Signed=&SigningBonusMin=0&SigningBonusMax=0&PlayerName=&OverallNumber=0'
        browser.visit(url)
        browser.reload()
        
        sleep(2)
        
        
        try:
            while browser.find_by_css('button').last.text == 'SEE MORE PLAYERS':
                browser.find_by_css('button').last.click()
                sleep(.5)
        except:
            print('caught error')

        soup = BeautifulSoup(browser.html, 'html.parser')
        table_data = soup.find_all('td')
        table_headers = soup.find_all('th')[0:12]

        #gets column names from table_headers on requested page
        column_names = []
        for x in table_headers:
            column_names.append(x.text.strip())

        df = pd.DataFrame(columns = column_names, index = [i for i in range(int(len(table_data)/12)+1)])
        #loops through requested data and appends it to table in a not so smart manner
        cnt = -1
        try:
            for i in range((len(table_data))):
                if(i % len(table_headers) == 0):
                    cnt += 1
                if 'colspan' in str(table_data[i]):
                    table_data.pop(i)
                if i % 12 == 3:
                    df.iat[cnt, i % len(table_headers)] = table_data[i].contents[0]['alt']
                else:
                    df.iat[cnt, i % len(table_headers)] = table_data[i].text.strip()
        except:
            print(f'done: {year}')
        df = df.drop(columns = 'Reports')
        df.to_csv(f'mlb_draft_player_salaries_{year}.csv',index=False)

In [ ]:
years = [i for i in range(2009, 2019)]
scrape_mlb(years)

starting: 2009
done: 2009
starting: 2010
done: 2010
starting: 2011
done: 2011
starting: 2012
done: 2012
starting: 2013
done: 2013
starting: 2014
done: 2014
starting: 2015
done: 2015
starting: 2016
done: 2016
starting: 2017


In [46]:
browser = Browser('chrome')

In [47]:
year = '2010'
url = f'https://www.baseballamerica.com/draft-history/mlb-draft-database/#/?Year={year}&Round=0&TeamId=0&Position=&SchoolName=&FourYearSchoolType=false&JuniorCollegeType=false&HighSchoolType=false&OtherSchoolType=false&StateOrCountry=&Signed=&SigningBonusMin=0&SigningBonusMax=0&PlayerName=&OverallNumber=0'
browser.visit(url)

In [48]:
while browser.find_by_css('button').last.text == 'SEE MORE PLAYERS':
    try:
        browser.find_by_css('button').last.click()
    except:
        sleep(.5)

In [49]:
soup = BeautifulSoup(browser.html, 'html.parser')
table_data = soup.find_all('td')
table_headers = soup.find_all('th')[0:12]

In [50]:
column_names = []
for x in table_headers:
    column_names.append(x.text.strip())


In [51]:
df = pd.DataFrame(columns = column_names, index = [i for i in range(int(len(table_data)/12)+1)])
len(table_headers)

12

In [84]:
#loops through requested data and appends it to table in not so smart manner
cnt = -1
try:
    for i in range((len(table_data))):
        if(i % len(table_headers) == 0):
            cnt += 1
        if 'colspan' in str(table_data[i]):
            table_data.pop(i)
#         try:
#             df.iat[cnt, i % len(table_headers)] = table_data[i].contents[0]['alt']
#         except TypeError:
#             df.iat[cnt, i % len(table_headers)] = table_data[i].text.strip()
        if i % 12 == 3:
            df.iat[cnt, i % len(table_headers)] = table_data[i].contents[0]['alt']
        else:
            df.iat[cnt, i % len(table_headers)] = table_data[i].text.strip()
except:
    print(f'done: {year}')

In [77]:
if table_data[15].contents[0]['alt']:
    print(1)

1


In [83]:
table_data[27].contents[0]['alt']

'Orioles'

In [85]:
df

,Year,RD,Pick,Team,Pos,Player,School,Type,ST,Signed,Bonus,Reports
0,2010,1,1,Nationals,OF,Bryce Harper,JC of Southern Nevada,JC,NV,Y,"$6,250,000",Scouting\n BA
1,2010,1,2,Pirates,RHP,Jameson Taillon,The Woodlands (Texas) HS,HS,TX,Y,"$6,500,000",Scouting\n BA
2,2010,1,3,Orioles,SS,Manny Machado,Brito Miami Private HS,HS,FL,Y,"$5,250,000",Scouting\n BA
3,2010,1,4,Royals,SS,Christian Colon,Cal State Fullerton,4YR,CA,Y,"$2,750,000",Scouting\n BA
4,2010,1,5,Indians,LHP,Drew Pomeranz,Mississippi,4YR,MS,Y,"$2,650,000",Scouting\n BA
5,2010,1,6,Diamondbacks,RHP,Barret Loux,Texas A&M,4YR,TX,N,,Scouting\n BA
6,2010,1,7,Mets,RHP,Matt Harvey,North Carolina,4YR,NC,Y,"$2,525,000",Scouting\n BA
7,2010,1,8,Astros,2B,Delino DeShields,"Woodward Academy, College Park, Ga.",HS,GA,Y,"$2,150,000",Scouting\n BA
8,2010,1,9,Padres,RHP,Karsten Whitson,Chipley (Fla.) HS,HS,FL,N,,Scouting\n BA
9,2010,1,10,Athletics,OF,Michael Choice,Texas-Arlington,4YR,TX,Y,"$2,000,000",Scouting\n BA
